In [ ]:
!pip install ultralytics
!pip install mediapipe
!pip install scikit-learn
!pip install joblib
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 94.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C


In [3]:
# Cloning and installing YOLOv5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
%pip install -r requirements.txt --quiet

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [1]:
import os
import cv2
import joblib
import torch
import mediapipe as mp
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [4]:
import cv2
import numpy as np
import torch
import mediapipe as mp
from pathlib import Path
from models.common import DetectMultiBackend
from utils.general import non_max_suppression
from utils.torch_utils import select_device

# Load YOLOv5 model
yolo_model_path = '/content/best.pt'  # your pretrained model path
device = select_device('')
yolo_model = DetectMultiBackend(yolo_model_path, device=device)
yolo_model.warmup(imgsz=(1, 3, 640, 640))

# Load MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

# Extract YOLO features: return 1-hot for detected class
def extract_yolo_feature(img, num_classes=26):
    try:
        img_resized = cv2.resize(img, (640, 640))
        img_rgb = img_resized[..., ::-1].transpose((2, 0, 1))
        img_rgb = np.ascontiguousarray(img_rgb)
        img_tensor = torch.from_numpy(img_rgb).to(device).float() / 255.0
        img_tensor = img_tensor.unsqueeze(0)

        pred = yolo_model(img_tensor, augment=False, visualize=False)
        pred = non_max_suppression(pred, conf_thres=0.5)[0]

        vec = np.zeros(num_classes)
        if pred is not None and len(pred):
            cls_id = int(pred[0][5].item())
            vec[cls_id] = 1
        return vec
    except:
        return np.zeros(num_classes)

# Extract MediaPipe features
def extract_mediapipe_feature(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    if results.multi_hand_landmarks:
        landmarks = []
        for lm in results.multi_hand_landmarks[0].landmark:
            landmarks.extend([lm.x, lm.y, lm.z])
        return np.array(landmarks)
    return np.zeros(63)  # 21 landmarks * 3



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 v7.0-425-g85acef3a Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7313943 parameters, 0 gradients


In [5]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Define the dataset directory and label set (A-Z)
dataset_path = '/content/drive/MyDrive/Train_Alphabet'
labels = [chr(i) for i in range(ord('A'), ord('Z')+1)]

# Feature and label storage
X, y = [], []

# Iterate through label folders
for label_idx, label in enumerate(labels):
    folder = os.path.join(dataset_path, label)
    if not os.path.isdir(folder):
        print(f"❌ Skipping missing folder: {label}")
        continue

    image_files = os.listdir(folder)
    print(f"🔍 Processing {label} ({len(image_files)} images)")

    for fname in image_files:
        path = os.path.join(folder, fname)
        img = cv2.imread(path)

        if img is None:
            print(f"⚠️ Skipping unreadable image: {path}")
            continue

        # Extract fused features from YOLO + MediaPipe
        try:
            yolo_feat = extract_yolo_feature(img)
            mp_feat = extract_mediapipe_feature(img)
            fused_feat = np.concatenate([yolo_feat, mp_feat])
        except Exception as e:
            print(f"⚠️ Feature extraction failed for {fname}: {e}")
            continue

        X.append(fused_feat)
        y.append(label_idx)

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

print(f"\n✅ Feature extraction complete.")
print(f"📊 Total samples: {len(X)}, Feature dimension: {X.shape[1]}")


🔍 Processing A (920 images)


KeyboardInterrupt: 

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Train Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save the trained model
import joblib
joblib.dump(clf, '/content/early_fusion_model.pkl')
print("✅ Model trained and saved to '/content/early_fusion_model.pkl'")

In [6]:
import os
import cv2
import numpy as np
import torch
import joblib
import mediapipe as mp
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, confusion_matrix
from utils.general import non_max_suppression
from utils.torch_utils import select_device
from models.common import DetectMultiBackend
import seaborn as sns
import matplotlib.pyplot as plt

#  Loading trained early-fusion classifier
clf = joblib.load('/content/early_fusion_model.pkl')  # path to trained model

# Set up YOLOv5
yolo_model_path = '/content/best.pt'
device = select_device('')
yolo_model = DetectMultiBackend(yolo_model_path, device=device)
yolo_model.warmup(imgsz=(1, 3, 640, 640))

# Set up MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

# Feature Extraction Functions
def extract_yolo_feature(img, num_classes=26, conf_threshold=0.6):
    try:
        img_resized = cv2.resize(img, (640, 640))
        img_rgb = img_resized[..., ::-1].transpose((2, 0, 1))  # BGR->RGB, HWC->CHW
        img_rgb = np.ascontiguousarray(img_rgb)
        img_tensor = torch.from_numpy(img_rgb).to(device).float() / 255.0
        if img_tensor.ndimension() == 3:
            img_tensor = img_tensor.unsqueeze(0)

        pred = yolo_model(img_tensor)
        pred = non_max_suppression(pred, conf_thres=conf_threshold)[0]

        vec = np.zeros(num_classes, dtype=np.float32)
        if pred is not None and len(pred):
            class_id = int(pred[0][5].item())
            if 0 <= class_id < num_classes:
                vec[class_id] = 1.0
        return vec
    except Exception:
        return None

def extract_mediapipe_feature(img):
    try:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            features = []
            for lm in results.multi_hand_landmarks[0].landmark:
                features.extend([lm.x, lm.y, lm.z])
            return np.array(features, dtype=np.float32)  # 63 elements
        else:
            return None
    except Exception:
        return None

#  Inference on test dataset
test_path = '/content/drive/MyDrive/Test_Alphabet'
labels = [chr(i) for i in range(ord('A'), ord('Z')+1)]
confidence_threshold = 0.6

y_true, y_pred = [], []

for label_idx, label in enumerate(labels):
    folder = os.path.join(test_path, label)
    if not os.path.isdir(folder):
        continue

    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)
        img = cv2.imread(path)
        if img is None:
            continue

        try:
            yolo_feat = extract_yolo_feature(img, conf_threshold=confidence_threshold)
            mp_feat = extract_mediapipe_feature(img)

            if yolo_feat is None or mp_feat is None:
                pred = -1  # mark failure
            else:
                fused_feat = np.concatenate([yolo_feat, mp_feat])  # 26 + 63 = 89 features
                pred = clf.predict([fused_feat])[0]
        except Exception as e:
            print(f" Feature extraction failed for {fname}: {e}")
            pred = -1

        y_true.append(label_idx)
        y_pred.append(pred)

# Original Report
total = len(y_true)
correct = sum(yt == yp for yt, yp in zip(y_true, y_pred))
accuracy = correct / total * 100 if total > 0 else 0.0

valid_idx = [i for i, p in enumerate(y_pred) if p != -1]
y_true_valid = [y_true[i] for i in valid_idx]
y_pred_valid = [y_pred[i] for i in valid_idx]

if len(valid_idx) > 0:
    print("\n Classification Report (valid predictions only):")
    print(classification_report(y_true_valid, y_pred_valid, target_names=labels, zero_division=0))
else:
    print(" No valid predictions.")

print(f"\n Total Samples: {total}")
print(f" Failed samples (treated as incorrect): {total - len(valid_idx)}")
print(f" Overall Accuracy (including failures): {accuracy:.2f}%")

#  Evaluation: Accuracy, Macro P/R/F1, Confusion Matrix (A–Z)
acc_overall = accuracy_score(y_true, y_pred) if total > 0 else 0.0
print("\n=== Overall Accuracy (A–Z; failures counted as incorrect) ===")
print(f"Accuracy: {acc_overall * 100:.2f}%")

prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true,
    y_pred,
    labels=list(range(len(labels))),   # only A–Z classes
    average='macro',
    zero_division=0
)
print("\n=== Macro-Averaged Metrics (A–Z only) ===")
print(f"Precision (Macro): {prec_macro:.4f}")
print(f"Recall (Macro):    {rec_macro:.4f}")
print(f"F1-Score (Macro):  {f1_macro:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred, labels=list(range(len(labels))))
plt.figure(figsize=(16, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix — ASL Letters (Early Fusion)')
plt.tight_layout()
plt.savefig('/content/confusion_matrix_early_fusion_letters.png', dpi=200, bbox_inches='tight')
plt.show()
print(" Saved confusion matrix to: /content/confusion_matrix_early_fusion_letters.png")


YOLOv5 🚀 v7.0-425-g85acef3a Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7313943 parameters, 0 gradients



 Classification Report (valid predictions only):
              precision    recall  f1-score   support

           A       1.00      0.99      0.99        92
           B       1.00      1.00      1.00        89
           C       1.00      1.00      1.00        92
           D       0.99      1.00      0.99        82
           E       1.00      1.00      1.00        76
           F       0.99      1.00      0.99        90
           G       1.00      1.00      1.00        98
           H       1.00      0.99      0.99        97
           I       1.00      1.00      1.00        91
           J       1.00      1.00      1.00        91
           K       0.99      0.99      0.99        92
           L       1.00      1.00      1.00        93
           M       1.00      1.00      1.00        85
           N       1.00      1.00      1.00        93
           O       0.99      1.00      0.99        96
           P       0.99      1.00      0.99        91
           Q       1.00      0.